In [147]:
from multiprocessing.sharedctypes import Value
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from pandas import value_counts
from scipy import ndimage
from skimage import io,color,measure
import random 

In [148]:
Nodes=([])
i=1
M = 800 #no. of pixel in column  
N = 800 #no. of pixel in row 
for x in range(M,-2 , -2):
    if i%2==0:
        for y in range(2,N+2 , 4):
            Nodes.append([y,x])
    else:
        for y in range(0,N+4 , 4):
            Nodes.append([y,x])
    i=i+1

# for x in range(2,3000 , 4):
#     for y in range(2,3000 , 4):
#         Nodes.append([x,y])
Points=np.array(Nodes)
# number of nodes
NON=(int)(Points.size/2)
print(NON)
print(Points)

80401
[[  0 800]
 [  4 800]
 [  8 800]
 ...
 [792   0]
 [796   0]
 [800   0]]


In [149]:
p = int(N/4) #(no. of column) 200
m = int(M/4) #(no. of rows) 200
NOE=4*p*m #(number of elements) 160000
NPE=3 #(nodes per element)

EL=np.zeros([NOE,NPE])
k=0
for i in range(1,(p*m)+1):
    if (i%p==1)|(p==1):
        k=k+1

# for the first element        
    EL[2*(i-1)*2,0]=k*(p+1)+i
    EL[2*(i-1)*2,1]=k*(p+1)+i-p
    EL[2*(i-1)*2,2]=EL[2*(i-1)*2,1]-1
# for the second element 
    EL[2*(i-1)*2+1,0]=k*(p+1)+i
    EL[2*(i-1)*2+1,1]=k*(p+1)+i+p+1
    EL[2*(i-1)*2+1,2]=k*(p+1)+i-p
# for the third element  k*p+1+i
    EL[2*(i-1)*2+2,0]=k*(p+1)+i
    EL[2*(i-1)*2+2,1]=k*(p+1)+i+p
    EL[2*(i-1)*2+2,2]=EL[2*(i-1)*2+2,1]+1
# for the fourth element  
    EL[2*(i-1)*2+3,0]=k*(p+1)+i
    EL[2*(i-1)*2+3,1]=k*(p+1)+i-p-1
    EL[2*(i-1)*2+3,2]=k*(p+1)+i+p

In [150]:
EL

array([[2.0200e+02, 2.0000e+00, 1.0000e+00],
       [2.0200e+02, 4.0300e+02, 2.0000e+00],
       [2.0200e+02, 4.0200e+02, 4.0300e+02],
       ...,
       [8.0200e+04, 8.0401e+04, 8.0000e+04],
       [8.0200e+04, 8.0400e+04, 8.0401e+04],
       [8.0200e+04, 7.9999e+04, 8.0400e+04]])

In [151]:
img=cv.imread('Circular_microstructure_color_10.jpg')
img1 = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
ret, thresh2 = cv.threshold(img1, 120, 255, cv.THRESH_BINARY_INV)
cv.imwrite('gray_ms.jpg',thresh2)
#cv.imshow()
#cv.waitKey(0)

True

In [152]:
# Segregation of triangular element # refer sheet3 of open cv doubt excel sheet 
EL = EL.astype(int)
set1=([])
set2=([])
set1n=0
set2n=0
for i in range(0,NOE):
    k=0
    for j in range(0,3):
        a=Points[EL[i][j]-1]
        if a[0]==N:
            a[0]=N-1
        if a[1]==0:
            a[1]=1
        if thresh2[abs(a[1]-M)][a[0]]==0:   ### for indexing in open cv pls refer  https://datahacker.rs/how-to-access-and-edit-pixel-values-in-opencv-with-python/
            k=k+1
        if a[0]==N-1:
            a[0]=N
        if a[1]==1:
            a[1]=0
    if k==3:
        set1.append(i+1)
        set1n=set1n+1
    else:
        set2.append(i+1)
        set2n=set2n+1

In [153]:
# My addition (Boundary sets)
YTop=([])
YBot=([])
XBack=([])
XFront=([])
TopRight=([p+1])
TopLeft=([1])
BotRight=([NON])
BotLeft=([NON-p])
delRow=NON-(p+1)
delCol=p
for i in range(2,p+1):
    YTop.append(i)
    YBot.append(delRow+i)
CommonDiff=2*p+1
for i in range(1,NON-p,CommonDiff):
    if i==1:
        continue
    XBack.append(i)
    XFront.append(delCol+i)
# print(Corners)
# print(YTop)
# print(YBot)
# print(XBack)
# print(XFront)

In [ ]:
output_file = open('Abaqus5001.INP','w')
output_file.write('*Heading\n')
output_file.write('**Preprint, echo=NO, model=NO, history=NO, contact=NO \n')
output_file.write('*Part, name=Part-1 \n')
output_file.write('*Node \n')
k=0
nor=0.00125
for i in Points: # nodes
     k=k+1
     output_file.write(str(k),)
     output_file.write(',')
     output_file.write(str(round(i[0]*nor,5)),)
     output_file.write(',')
     output_file.write(str(round(i[1]*nor,5))+'\n')

output_file.write('*Element, type = CPS3 \n')
k=0
for i in EL: 
     k=k+1
     output_file.write(str(k),)
     output_file.write(',')
     output_file.write(str(i[0]),)
     output_file.write(',')
     output_file.write(str(i[1]),)
     output_file.write(',')
     output_file.write(str(i[2])+'\n')
output_file.write('*Elset, elset=Set-1 \n')
k=0
for i in set1:

     output_file.write(str(i),)
     output_file.write(',')
     k=k+1
     if k%6==0:
          output_file.write('\n')
output_file.write('\n')
output_file.write('*Elset, elset=Set-2 \n')
k=0
for i in set2:

     output_file.write(str(i),)
     output_file.write(',')
     k=k+1
     if k%6==0:
          output_file.write('\n')


output_file.write('*Nset, nset=YTop \n')
k = 0
for i in YTop:
    output_file.write(str(i))
    output_file.write(',')
    k += 1
    if k % 6 == 0:
        output_file.write('\n')
output_file.write('\n')  # optional newline for separation

output_file.write('*Nset, nset=YBot \n')
k = 0
for i in YBot:
    output_file.write(str(i))
    output_file.write(',')
    k += 1
    if k % 6 == 0:
        output_file.write('\n')
output_file.write('\n')  # optional newline for separation

output_file.write('*Nset, nset=XBack \n')
k = 0
for i in XBack:
    output_file.write(str(i))
    output_file.write(',')
    k += 1
    if k % 6 == 0:
        output_file.write('\n')
output_file.write('\n')  # optional newline for separation

output_file.write('*Nset, nset=XFront \n')
k = 0
for i in XFront:
    output_file.write(str(i))
    output_file.write(',')
    k += 1
    if k % 6 == 0:
        output_file.write('\n')
output_file.write('\n')  # optional newline for separation

output_file.write('*Nset, nset=TopLeft \n')
k = 0
for i in TopLeft:
    output_file.write(str(i))
    output_file.write(',')
    k += 1
    if k % 6 == 0:
        output_file.write('\n')
output_file.write('\n')  # optional newline for separation

output_file.write('*Nset, nset=TopRight \n')
k = 0
for i in TopRight:
    output_file.write(str(i))
    output_file.write(',')
    k += 1
    if k % 6 == 0:
        output_file.write('\n')
output_file.write('\n')  # optional newline for separation

output_file.write('*Nset, nset=BotLeft \n')
k = 0
for i in BotLeft:
    output_file.write(str(i))
    output_file.write(',')
    k += 1
    if k % 6 == 0:
        output_file.write('\n')
output_file.write('\n')  # optional newline for separation

output_file.write('*Nset, nset=BotRight \n')
k = 0
for i in BotRight:
    output_file.write(str(i))
    output_file.write(',')
    k += 1
    if k % 6 == 0:
        output_file.write('\n')
output_file.write('\n')  # optional newline for separation         

output_file.write('*Solid Section, elset=Set-1, material=Material-1 \n')
output_file.write(', \n')
output_file.write('*Solid Section, elset=Set-2, material=Material-2 \n')
output_file.write(', \n')
output_file.write('*End Part \n')

output_file.write('**Assembly \n')
output_file.write('*Assembly, name=Assembly \n')
output_file.write('*Instance, name=Part-1-1, part=Part-1 \n')
output_file.write('*End Instance \n')

for xf, xb in zip(XFront, XBack):
    output_file.write('**Constraint: EqnXu\n')
    output_file.write('*Equation\n')
    output_file.write('3\n')
    output_file.write(f'Part-1-1.{xf}, 1, 1.\n')
    output_file.write(f'Part-1-1.{xb}, 1, -1.\n')
    output_file.write(f'Part-1-1.{BotRight[0]}, 1, -1.\n')  # Only one node in BotRight

for xf, xb in zip(XFront, XBack):
    output_file.write('**Constraint: EqnXv\n')
    output_file.write('*Equation\n')
    output_file.write('2\n')
    output_file.write(f'Part-1-1.{xf}, 2, 1.\n')
    output_file.write(f'Part-1-1.{xb}, 2, -1.\n')

for yt, yb in zip(YTop, YBot):
    output_file.write('**Constraint: EqnYv\n')
    output_file.write('*Equation\n')
    output_file.write('3\n')
    output_file.write(f'Part-1-1.{yt}, 2, 1.\n')
    output_file.write(f'Part-1-1.{yb}, 2, -1.\n')
    output_file.write(f'Part-1-1.{TopLeft[0]}, 1, -1.\n')  # Only one node in TopLeft

for yt, yb in zip(YTop, YBot):
    output_file.write('**Constraint: EqnYu\n')
    output_file.write('*Equation\n')
    output_file.write('2\n')
    output_file.write(f'Part-1-1.{yt}, 1, 1.\n')
    output_file.write(f'Part-1-1.{yb}, 1, -1.\n')


# TOPRIGHT PE CONDITIONS (OPTIONAL)
output_file.write('**Constraint: TopRighty\n')
output_file.write('*Equation\n')
output_file.write('4\n')
output_file.write(f'Part-1-1.{TopRight[0]}, 2, 1.\n')
output_file.write(f'Part-1-1.{BotRight[0]}, 2, -1.\n')
output_file.write(f'Part-1-1.{TopLeft[0]}, 2, -1.\n')
output_file.write(f'Part-1-1.{BotLeft[0]}, 2, +1.\n')

output_file.write('**Constraint: TopRightx\n')
output_file.write('*Equation\n')
output_file.write('2\n')
output_file.write(f'Part-1-1.{TopRight[0]}, 1, 1.\n')
output_file.write(f'Part-1-1.{TopLeft[0]}, 1, -1.\n')
output_file.write(f'Part-1-1.{BotRight[0]}, 1, -1.\n')
output_file.write(f'Part-1-1.{BotLeft[0]}, 1, 1.\n')

output_file.write('*End Assembly \n')

output_file.write('**Materials \n')
output_file.write('*Material, name = Material-1 \n')
output_file.write('*Elastic \n')
output_file.write('210e+3,0.3 \n')
output_file.write('*Material, name = Material-2 \n')
output_file.write('*Elastic \n')
output_file.write('150e+3,0.2 \n')

output_file.write('*Step, name=ApplyPBCs, nlgeom=NO\n')
output_file.write('*Static\n')
output_file.write('1., 1., 1e-05, 1.\n')

output_file.write('**Output, field, variable=PRESELECT \n')
output_file.write('**Output, history, variable=PRESELECT \n')
output_file.write('*End Step \n')
output_file.close()
